# Working with Sockets in Python

## What is an Socket?

Socket programming is a way of connecting two nodes on a network to communicate with each other. One socket(node) listens on a particular port at an IP, while the other socket reaches out to the other to form a connection. The server forms the listener socket while the client reaches out to the server. 

In Python, you'll be creating local `socket` objects from the `socket` module that serve as proxies for interacting with the operation system and the sockets it creates.

The below code snippet shows the basics of creating a local socket.

In [ ]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(type(s))
print(s)

### Troubleshooting

Throughout this workbook you may find that there are problems with sockets being taken up or your code leaving sockets running. One place to start if you can't figure out how to fix things is to restart the Jupyter Notebook Kernel. If you're using the web interface, you will find that option in the drop down menu under 'Kernel' at the top of the page.

### Netcat Setup

For this workbook you will need to set up connections outside of your computer, I strongly recommend using netcat for this. Most linux flavors will come with netcat as a command line binary. For Windows, you can download it from [this page](https://nmap.org/download.html). It will be one of the files inside the zipped folder, you can then run it using the Windows command line.

If you just want a connection you can type into and receive things from use:
`nc 127.0.0.1 6789`
Or for receiving connections:
`nc -l 7654`

If you need to push something to a connection, you can use the following syntax:  
`"hello world" | nc localhost 12345`  
If you need to listen for an incomming connection on a specific port and serve text to it, you can use the `-l` flag:  
`"hello world" | nc -l 12345`  

You will definitely need to play around with the tool a little more if this is your first time using it. Don't hesitate to search for answers or ask for help!

### Connecting a Socket

We'll start by working through creating a client socket and sending data.

You'll need to use the `connect`, `sendall`, and `recv` methods.

Note: Sockets use the `Bytes` data structure instead of `String`s. They look similar in code, bytes are differentiated from strings by having a `b` character before the initial double or single quotation marks. Bytes store exactly represent the bytes the are stored as on the local machine, which makes them preferable for sending data over a network. For the purposes of this class that shouldn't be too much of an issue, but that's why you'll see lots of `b`s floating around in the code and output.

First, set up a local listening conneciton using netcat at the port address 65432 sending data:  
`"Hello right back" | netcat -l 65432`

Then run the below code block. You should see the string sent by netcat printed here, and on the command line you should see the string you send from this notebook!

In [ ]:
import socket

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)

print('Received', repr(data))

## Exercise: Connecting to your computer

1. Connect to port 5555 locally, send the message "How do you do?" and receive back and print the message "I'm good, what about you?"

2. Connect to port 5678, send the message "1", receive the message "2", then send the message "3", and receive the message "4"

Hint: You'll probably need switch over to your netcat listener to send the messages back and forth. It might help to include newlines at the end of the messages you send.

3. Connect to port 9876, receive the message "1", send the message "2", then receive the message "3", and send the message "4"

4. Connect to port 8765, receive a message, and then send back the message you received twice with the string "doubled:" before it. So if you received 'hello', send back 'doubled:hellohello'

5. Connect to a website of your choice and send them the string `GET / HTTP/1.0\n\n`. You should get an HTTP response back.

## Listening on a socket

Sometimes you don't want to go out and get a conneciton. Sometimes you want someone else to connect to you first. That's what listening sockets are for.

For listening, you'll use the `bind`, `listen`, and `accept` methods for a socket. `bind` takes a tuple of an ip address and port. The port is where it listens. The ip address is a bit  more complex. For this course we'll mostly be binding using a local host address `127.0.0.1`, but if your operating system is using multiple addresses you specificy which one you want. An empty string and `0.0.0.0` both indicate to bind to all addresses.

After listen is called, you can call `accept` to accept an incoming connection. This returns a tuple of the connection (which is a new sockets that can be used the same as your connection socket from before) and another tuple of the ip address and port that connected to your listening port.

The below code snippet runs an echo server. Connect to it using netcat and send some messages.

In [ ]:
import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 34567        # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            print(data)
            if not data:
                break
            conn.sendall(data)

You should see that each time you sent a message, it was printed here and send back to your netcat connection.

## Exercise: Listening for Connections

1. Create a listening socket on port 7890 that listens for incoming connections, and then sends those connecitons back their IP address and host name

2. Create a listening socket on port 8675 that listens for incoming connections, asks the connection for a phrase, and then sends back how many letters are in that phrase

3. Create a listening socket on port 3456 that listens for incoming connections, asks it to send a first number, asks for a second number, and then sends the sum of those numbers

4. Create a listening socket on port 1248 that listens for incoming connections, and then echos the messages sent to it except for the message "exit". If it received the message "exit" it prompts for exit to be send one more time, in which case it closes, otherwise it goes back to echoing as normal.